# Make schedule



In [ ]:
from jupyterinstruct.webtools import makedateschedule

makedateschedule('./cmse401-S21-student/assignments/')


## CSV file date based migration (this worked well)

It is currently turned off.  Change False to True in last line.

The csv file should have two columns ```['Current File', 'Code']```. Where ```Current File``` is the current filename and the ```Code``` is the new date code in the format ```MMDD```.  Running this code will covert from the current file to a filename with the new code and update all student links. Similar to this:

https://docs.google.com/spreadsheets/d/19dYzkAz2E2GbpvtNIfLbfrsQxz5PVyznKOSzwE2pQVw/edit#gid=2142090757

In [ ]:
csvfile = 'CMSE401_F20-S21_Migration_file.csv'

In [ ]:
from jupyterinstruct import InstructorNotebook 
from pathlib import Path
import pandas

df = pandas.read_csv(csvfile)
df = df[['Current File', 'Code']]
df['Code'] = df['Code'].apply(lambda x: str(x).zfill(4))

for i, row in df.iterrows():
    filename = row['Current File']
    date = row['Code']
    if isinstance(filename,str):
        if filename[:4] == date:
            print(f" No change to {filename} to {row['Code']}")
        else:
            print(f" Fixing {filename} to {row['Code']}")
            InstructorNotebook.changeprefix(filename, row['Code'], False, False)

## Fixing Header/Footer 
The following will go though the entire directory and try to strip out Headers and footers and replace them with headers and footers in the following files:

    - In-class-Header.ipynb
    - In-class-Footer.ipynb
    - pre-class-Header.ipynb
    - Pre-class-Footer.ipynb
    
You may want to add some extra "REMOVE" functions based on your needs.



In [ ]:
from jupyterinstruct import InstructorNotebook as inb
from jupyterinstruct.nbfilename import nbfilename
from IPython.core.display import Javascript, HTML

def replease_head_foot(filename = ""):
    nbfile = nbfilename(filename)
    if not filename == str(nbfile):
        print(f"Instructor file not complient - Changing from {filename} to {str(nbfile)}")
        jupytermigrate.renamefile(filename, str(nbfile),True)
        
    filename=str(nbfile)
    
    try:
        nb = inb.InstructorNB(filename=filename)
    except Exception as e:
        print(f"ERROR: {e}\n\n\n")
        return 1
    
    nb.removebefore('ENDHEADER')
    nb.removeafter('STARTFOOTER')

    if (len(nb.contents.cells) == 0):
        print(f"ERROR {filename} - We Removed too much")
    else:

        if nbfile.isInClass:
            print('in-class')
            nb.headerfooter(headerfile='In-class-Header.ipynb', footerfile='In-class-Footer.ipynb')
        elif nbfile.isPreClass:
            print('pre-class')
            nb.headerfooter(headerfile='Pre-class-Header.ipynb', footerfile='Pre-class-Footer.ipynb')


        #COMMENT OUT TO DISABLE WRITING (For testing)
        nb.writenotebook(str(nbfile))

        # Make a link for review
        display(
           HTML(f"<a href={filename} target=\"blank\">{filename}</a>"))



In [ ]:
from pathlib import Path
from jupyterinstruct import InstructorNotebook as inb
from jupyterinstruct.nbfilename import nbfilename

directory = Path('.')
allnbfiles = sorted(directory.glob('*.ipynb'))

for filename in allnbfiles:
    nbfile = nbfilename(filename)
    if nbfile.prefix.isdigit():
        if int(nbfile.prefix[:4]) > 0:
            print(nbfile)
            replease_head_foot(str(nbfile))
        

## Check and Fix filenames in current directory

This is a nice thing to do at the beginning of a semester to make sure names have follow the described requirements. 

In [ ]:
from pathlib import Path
from jupyterinstruct import InstructorNotebook as inb
from jupyterinstruct.nbfilename import nbfilename

directory = Path('.')
allnbfiles = sorted(directory.glob('*.ipynb'))

for filename in allnbfiles:
    nbfile = nbfilename(filename)
    if nbfile.isInstructor:
        if str(filename) != str(nbfile):
            inb.renamefile(str(filename), str(nbfile),True, True)

## Make a date schedule

https://docs.google.com/spreadsheets/d/19dYzkAz2E2GbpvtNIfLbfrsQxz5PVyznKOSzwE2pQVw/edit#gid=2142090757

In [ ]:
from jupyterinstruct.nbfilename import nbfilename
from pathlib import Path
from IPython.display import Markdown



student_folder = "."
S_path = Path(student_folder)
StudentFiles = S_path.glob(f"*.ipynb")

nameset = set()
for file in StudentFiles:
    nbfile = nbfilename(file)
    if nbfile.isDate:
        nameset.add(str(nbfile))
        
I_path = Path('.')
InstructorFiles = I_path.glob(f"*.ipynb")

schedule =  "| Date | Type | number | Topic | notebook |\n"
schedule += "|------|------|--------|-------|----------|\n"

for file in sorted(InstructorFiles):
    nbfile = nbfilename(str(file))
    if(nbfile.isInstructor and nbfile.isDate):
        nbfile.isInstructor = False
        thisfile = str(nbfile)
        
        filetype = "In-Class Assignment"
        if nbfile.isPreClass:
            filetype = "Pre-Class Assignment"
        nbfile.isInClass = False
        nbfile.isPreClass = False
        
        if thisfile in nameset:
            schedule += f"|  {nbfile.getlongdate()}, 2021  | {nbfile.basename()[0:4]} | {filetype} | [{nbfile.basename()[5:]}]({student_folder}{thisfile[:-6]}.html) | [ipynb]({student_folder}{str(thisfile)}) |\n"
        else:
            schedule += f"| {nbfile.getlongdate()}, 2021   | {nbfile.basename()[0:4]} | {filetype} | {nbfile.basename()[5:]} |\n"
Markdown(schedule)

---